In [1]:
import os

import openai
from openai import OpenAI

# Part 1

In [2]:
client = OpenAI(api_key=os.environ["OPENAI_KEY"])

In [3]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "This is my test request to ChatGPT"
        }
      ]
    }
  ],
)

In [4]:
response.choices[0].message.content

'Thanks for reaching out! How can I assist you today?'

# Part 2

## Q1. Running Elastic

In [5]:
!curl elasticsearch:9200

{
  "name" : "760ff73ef145",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "tqFS-PexTHiPBpvAUEeFww",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Q2. Indexing the data

Which function do you use for adding your data to elastic?

* insert
* **index**
* put
* add

## Q3. Searching

In [6]:
from elasticsearch import Elasticsearch

In [7]:
index_name = "course-questions"
es_client = Elasticsearch('http://elasticsearch:9200') 

In [8]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                }
            },
        }
    }
}

In [9]:
response = es_client.search(index=index_name, body=search_query)

In [10]:
response.body["hits"]

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions',
   '_id': 'pX_FJ5ABo5uIm2EbifUu',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}}]}

## Q4. Filtering

In [11]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [12]:
response = es_client.search(index=index_name, body=search_query)

In [13]:
response.body["hits"]["hits"][2]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

## Q5. Building a prompt

In [14]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [15]:
def elastic_search(client, query, search_query):

    response = client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [16]:
def build_prompt(query, search_results):


    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    for doc in search_results:
        context += f"{context_template.format(question=doc["question"], text=doc["text"])}\n\n"
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
search_results = elastic_search(es_client, query, search_query)

In [18]:
prompt = build_prompt(query, search_results)

In [19]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [20]:
len(prompt)

1462

## Q6. Tokens

In [21]:
import tiktoken

In [22]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [23]:
len(encoding.encode(prompt))

322

In [24]:
encoding.encode(prompt)[:10]

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122]

In [25]:
[encoding.decode_single_token_bytes(token) for token in  encoding.encode(prompt)[:10]]

[b"You're",
 b' a',
 b' course',
 b' teaching',
 b' assistant',
 b'.',
 b' Answer',
 b' the',
 b' QUESTION',
 b' based']

## Bonus: generating the answer (ungraded)
 

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [27]:
result = llm(prompt)

In [28]:
print(result)

To execute a command in a running Docker container, you can use the `docker exec` command. First, find the `container-id` using the `docker ps` command. Then, execute your desired command in the specific container as follows:

1. List all running containers to find the `container-id`:
   ```sh
   docker ps
   ```

2. Execute a command in the running container using the `container-id`. For example, to start a bash session:
   ```sh
   docker exec -it <container-id> bash
   ```
Replace `<container-id>` with the actual ID of your container.


## Bonus: calculating the costs (ungraded)

In [29]:
input_cost = 150 * 1000 * 0.005 / 1000

In [30]:
output_cost = 250 * 1000 * 0.015 / 1000

In [31]:
print(f"Cost to run 1000 requests with 150 tokens and receive back 250 token: {input_cost + output_cost}$")

Cost to run 1000 requests with 150 tokens and receive back 250 token: 4.5$


In [32]:
prompt_token = len(encoding.encode(prompt))
result_token = len(encoding.encode(result))
input_cost =  prompt_token * 0.005 / 1000
output_cost = result_token * 0.015 / 1000

In [33]:
print(f"Cost to run 1 requests with {prompt_token} tokens and receive back {result_token} token: {round(input_cost + output_cost, 4)}$")

Cost to run 1 requests with 322 tokens and receive back 128 token: 0.0035$
